In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.style.use('default')

from z3 import *

import time
from os import listdir
import sys
sys.path.append("../common")

from utils import visualize, read_instance, save_solution, visualize_from_file, optimize

#reloads external modules when they are changed
%load_ext autoreload
%autoreload 2

# Try one instance

In [ ]:
ins_filename = "../instances/ins-1.txt"
#ins_filename = "ins-prova.txt"
W, n, rectangles = read_instance(ins_filename)
break_symmetries = True
allow_rotation = True

print("width: ", W)
print("number of rectangles: ", len(rectangles))
print("largest rectangle:", np.argmax([r.w * r.h for r in rectangles])+1)
print("break_symmetries:", break_symmetries)
print("")

try:
    H, positioned_rectangles = optimize('SAT', W, rectangles,
                                            break_symmetries = break_symmetries,
                                            allow_rotation = allow_rotation,
                                            verbose=True)
    plot = True
except Z3Exception as e:
    plot = False
    print("\n----Timeout----\n")

In [ ]:
w, h, dpi = 600, 600, 100
fig, ax = plt.subplots(figsize = (w/dpi, h/dpi), dpi = dpi)

fig, ax = visualize(W, H, positioned_rectangles, ax = ax)
fig.suptitle(f"{ins_filename.split('/')[-1].split('.')[0]}, {n} rectangles, W = {W}, H = {H}")
fig.tight_layout(pad = 1)
#plt.savefig("../" + ins_filename.split('.')[0])
plt.show()

# Compute all instances

In [ ]:
save_to_file = False

output_folder = "out/rotation/"
input_folder = "../instances/"

plot = True

break_symmetries = True
allow_rotation = True

print("")
print(50*"=")
for i in range(40):
    ins_filename = input_folder + f"ins-{i+1}.txt"
    solution_filename = output_folder + f"ins-{i+1}-sol.txt"
    W, n, rectangles = read_instance(ins_filename)

    m = np.argmax([r.w * r.h for r in rectangles])
    print(f"\nInstance {i+1}")
    print("Width:", W)
    print("Number of rectangles:", len(rectangles))
    print("Largest rectangle measures:", (rectangles[m].w, rectangles[m].h))
    print("Largest rectangle index:", m+1)
    print("Breaking symmetries:", break_symmetries)
    print("")

    execution_time = 0.
    H = 0
    try:
        start = time.time()
        H, positioned_rectangles = optimize('SAT', W, rectangles,
                                                break_symmetries = break_symmetries,
                                                allow_rotation = allow_rotation,
                                                verbose=True)
        execution_time = time.time() - start
        print("\nFound height:", H)
        print(f"Execution time in seconds: {execution_time:.3f}")

        if save_to_file:
            save_solution(solution_filename, W, H, positioned_rectangles)
            with open(solution_filename, "a") as f:
                f.write(f"\nexecution time in seconds: {execution_time:.3f}")
        if plot:
            w, h, dpi = 600, 600, 100
            fig, ax = plt.subplots(figsize = (w/dpi, h/dpi), dpi = dpi)

            fig, ax = visualize(W, H, positioned_rectangles, ax = ax)
            fig.suptitle(f"{ins_filename.split('/')[-1].split('.')[0]}, {n} rectangles, W = {W}, H = {H}")
            fig.tight_layout(pad = 1)
            #plt.savefig("../" + ins_filename.split('.')[0])
            plt.show()


    except Z3Exception as e:
        print("\n----Timeout----\n")
    
    print("")
    print(50*"=")

# Visualize all solutions

In [ ]:
sol_folder = "out/no_rotation/"
filenames = listdir(sol_folder)
instance_numbers = [int(name.split("-")[1]) for name in filenames]
_, filenames = zip(*sorted(zip(instance_numbers, filenames)))

plot_folder = "out/plots/"
for filename in filenames:
    instance_name = "-".join(filename.split(".")[0].split("-")[:-1])
    fig, ax = visualize_from_file(sol_folder + filename, plot_width = 900, dpi = 100, additional_info = instance_name)
    fig.savefig(plot_folder + filename.split(".")[0])
    plt.show()